# Welcome  

Notebook Author: Samuel Alter  
Notebook Subject: Capstone Project - Image Analysis

BrainStation Winter 2023: Data Science

This notebook is for running my images through deep learning networks. The images have been pre-processed in a separate notebook.

The goal is to find a pre-trained CNN to adapt to our specific use-case of finding insights about wildfire-prone landscapes.

## Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# import `ImageDataGenerator` to help facilitate 
# loading images directly from our computer

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Specify the dimensions for the target preprocess image size

height = 128 
width = 128
channels = 3

# Create training image data generator
train_datagen = ImageDataGenerator(rescale=1./255)

# Create validation image data generator
# Only apply rescaling to our validation data
validation_datagen = ImageDataGenerator(rescale=1./255)

# Specify where the images should be loaded from,
# as well as some additional attributes:
train_generator=train_datagen.flow_from_directory('/Users/psmith/data_sources/images/monkeys/train/',
                                                 target_size=(height,width),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical')

validation_generator=validation_datagen.flow_from_directory('/Users/psmith/data_sources/images/monkeys/validation/',
                                                 target_size=(height,width),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical')

In [2]:
from tensorflow.keras.applications import VGG19

height = 128 
width = 128
channels = 3

res_model = VGG19(weights='imagenet',
                   include_top=False,
                   input_shape=(height,width,channels))
res_model.summary()

80150528/80134624 [==============================] - 107s 1us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                            

In [ ]:
# Freeze all the layers in the base model
for layer in res_model.layers:
    layer.trainable = False